**How to use this notebook :**

This notebook intends to use a fine-tuned LLM version of LLama2 to answer questions students of UQAC may have about their school programs and classes.

- Run the load data to load Data from the github (alternatively, you can put whatever files, just create a folder called "data" in the colab session and add the files there (accepted extensions: txt, csv, doc, docx, xml, pdf, epub, hwp, ipynb, ppt))
- Run the setup which will install and import all required libraries and setup the LLM and the RAG model (loading the data files into the RAG model can take up to a couple dozens of minutes depending on the size of the data files)
- Do not run the "Test the LLM" part because it can take a few minutes and it is unnecessary (unless you want to see how it works)
- Run the Pipeline Setup
- You can then go to the Pipeline testing section and change the user_query to whatever you want and run the pipeline function

#**Load Data**

In [ ]:
!mkdir data
!cd data
!git init
!git remote add origin -f https://github.com/Giraud-Pierre/DeepLearning_FineTuneLLama2Project.git
!git sparse-checkout init --cone
!git sparse-checkout set data/RAG
!git pull origin main
!cd ../
!mv data/RAG/CyclesSupérieurs.txt data/CyclesSupérieurs.txt
!mv data/RAG/PremierCycles.txt data/PremierCycles.txt
!rmdir data/RAG
!rm data/README.md
!rm README.md

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Updating origin
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 148 (delta 45), reused 6 (delta 1), pack-reused 0
Receiving objects: 100% (148/148), 44.37 MiB | 8.00 MiB/s, done.
Resolving deltas: 100% (45/45), done.
From https://github.com/Giraud-Pierre/DeepLearning_FineTuneLLama2Project
 * [new branch]      main       -> origin/main
From https://github.co

#**Setup**

##Install All the Required Packages

In [ ]:
# Installs to run the LLM
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 26.9 MB/s eta 0:00:00


In [ ]:
#Installs to run the RAG model
!pip install -q llama-index llama-index-embeddings-huggingface auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

##Import All the Required Libraries

In [ ]:
#Import to run the LLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, PeftConfig
import torch


from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

##Import, setup and test the RAG model

In [ ]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
#Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# RAG configuration
Settings.llm = None
Settings.chunk_size = 1024 # fragment size (in chars) for the RAG
Settings.chunk_overlap = 256 # overlap between 2 adjacent fragment 4
                            # (so that no context is missing or not
                            # comprehensible because of the fragmentation)

LLM is explicitly disabled. Using MockLLM.


In [ ]:
documents = SimpleDirectoryReader("data").load_data()

print(len(documents)) #number of documents

# Vector storing the documents (can take up to a few dozens
# of minutes depending on the size of the documents)
index = VectorStoreIndex.from_documents(documents)

2


In [ ]:
# retriever configuration
top_k = 3 # number of fragments to get at each query
          # recommended: 2 to 4

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

# RAG query_engine configuration
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [ ]:
# RAG testing
query = "Quelles programmes proposent un cours sur les réseaux de neurones ?"
response = query_engine.query(query)

# Creating a context-like string that will receive the information from the RAG
# about the query
context = "Context:\n"
for item in response.source_nodes:
    # Show all the fragments selected by the RAG for being most likely
    # connected to the query
    context += item.text + "\n\n"

print(context)

Context:
Acqurir des connaissances fondamentales et appliques des rseaux de neurones et de l'apprentissage profond. Matriser l'laboration d'architectures complexes, leurs configurations et la slection des units/modules appropris selon le contexte.
Prsentations des principes thoriques du fonctionnement des neurones artificielles. Techniques et mthodologies lies  l'entranement des rseaux. Techniques du gradient et optimiseurs (SGD, Adam, RMSProp, Adagrad). Fonctions d'activation et hyperparamtres. Principales architectures de CNN, RNN, GAN, AE, etc. Units et modules spcialiss pooling, attention, transformeur, rservoir, LSTM, GRU. Autoapprentissage et espaces latents. Modlisation des donnes complexes pour les rseaux de neurones. Optimisation des hyperparamtres et valuation des modles entrans. Principales librairies pour l'apprentissage profond : Pytorch, Keras/Tensorflow ou autre.
Formule pdagogique : Cours Magistral
(01/2024)
APPARTENANCE DPARTEMENTALE
Informatique et mathmatique
PROGRAM

##Import and setup the LLM

In [ ]:
# bitsandbytes parameters
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
# load fine-tuned model from huggingFaces
model_name = "pirroflamme/Llama2_Finetuned_DeepLearning"
model = AutoModelForCausalLM.from_pretrained(model_name,
                  quantization_config=bnb_config,
                  device_map="auto"
)

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

#**Pipeline setup**

##Prompt template

In [ ]:
# prompt template (based on user query)
prompt_template_w_context = lambda context, prompt_engineering, user_query: f"""
[INST]
{context}

{prompt_engineering}

{user_query}
[/INST]
"""

##Context from RAG

In [ ]:
def get_context(user_query):
  response = query_engine.query(query)
  context = " "
  for item in response.source_nodes:
    # Show all the fragments selected by the RAG for being most likely
    # connected to the query
    context += item.text + "\n\n"

  return context

##Prompt_engineering

In [ ]:
prompt_engineering =""

In [ ]:
# Chain of thoughts (examples of correct question / answer)

#prompt_engineering += """ \n \n Exemples:
#Quels sont les cours disponible pour la Maîtrise en informatique ?
#answer: Les cours disponibles pour la maîtrise en informatique sont: Réseaux de neurones, Bases de données réparties, Vision artificielle et traitement des images, Informatique appliquée et optimisation, Structures de données avancées et leurs algorithmes, Interaction 3D et réalité virtuelle, Génie logiciel, Métaheuristiques en optimisation,  Architecture des applications d'entreprise, Intelligence d'affaires: principes et méthodes, Programmation sur architectures parallèles, Sécurité informatique, Systèmes intégrés de gestion d'entreprise, Gestion de projets informatiques, Programmation de plateformes mobiles, Fondamentaux de l'apprentissage automatique, Principes des moteurs de jeux, Cryptographie, Conception et architecture des systèmes d'infonuagique, Intelligence Artificielle, Gestion des incidents de sécurité, Sécurité des applications, Gestion de la cybersécurité et des données personnelles, Séminaire thématique en cybersécurité, Apprentissage profond, Séminaire thématique en Intelligence artificielle, Jeux sérieux, Séminaire thématique en jeu vidéo, Programmation réseau dans les jeux vidéo, Sécurité informatique pour l'Internet des Objets, Analyse forensique, Apprentissage automatique pour les données massives, Internet des objets, Atelier en optimisation avancée, Sujets spéciaux, Programmation objet avancée, Spécification, test et vérification, Principes de conception et de développement de jeux vidéo
#Quels programmes proposent le cours 8INF874 cryptographie ?
#answer: Les programmes proposant le cours cryptographie sont : le Programme court de deuxième cycle en informatique pour étudiants en séjour d'études, la Maîtrise en informatique (jeux vidéo), la Maîtrise en informatique (cybersécurité), la Maîtrise en informatique, la Maîtrise en informatique (professionnel), le Diplôme d'études supérieures spécialisées en informatique appliquée
#en quoi consiste le cours 8INF896 Séminaire thématique en intelligence artificielle ?
#answer : Le séminaire théùatique en intelligence artificielle permet de bénéficier d'une formation adaptée dans le domaine de l'intelligence artificielle. Le contenu est variable selon les besoins des étudiants et l'expertise professorale disponible.
#"""

# Does not work, completely breaks the model for some reason ...

In [ ]:
# Do not hallucinate
prompt_engineering += """\n Reste concis et factuel. Répond "je ne suis pas sûr de la réponse" si tu n'es pas sûr
"""

In [ ]:
prompt_engineering += "\n Répond à la question suivante en utilisant les informations ci-dessus si celles-ci sont utiles:"

In [ ]:
prompt_engineering += "\n \n \n \n"

## Pipeline function

In [ ]:
def pipeline(user_query, prompt_engineering):
  prompt = prompt_template_w_context(get_context(user_query),prompt_engineering, user_query)
  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

  response = tokenizer.batch_decode(outputs)[0]

  return response.split("en utilisant les informations ci-dessus si celles-ci sont utiles:")[1].replace("[/INST]", "\n Réponse: ")

#**Test the LLM without RAG**

In [ ]:
# Test the LLM without RAG

prompt = """[INST] Donne-moi le nom des cours sur l'apprentissage profond ? [/INST]""" # = User query

model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

<s> [INST] Donne-moi le nom des cours sur l'apprentissage profond ? [/INST] Les cours 'Apprentissage profond' se concentrent sur les théories de l'apprentissage. concrets. </s></s>


#**Pipeline Testing (LLM with RAG)**

In [ ]:
 # Change the user_query to your question
user_query = "Quelles sont les cours proposés sur le domaine de l'intelligence artificielle"
response= pipeline(user_query,prompt_engineering)

print(response)